In [5]:
from models.End_ExpansionNet_v2 import End_ExpansionNet_v2
from utils.image_utils import preprocess_image
from utils.language_utils import tokens2description
import pickle
import torch

In [12]:
from datasets import load_dataset

ds = load_dataset("Skyler215/KTVIC_Test_5_Captions")

c:\Anaconda\envs\exp3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Anaconda\envs\exp3\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\datasets--Skyler215--KTVIC_Test_5_Captions. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see t

In [20]:
from PIL import Image, ImageOps, ExifTags
ds['train']['img_new']

AttributeError: module 'PIL.Image' has no attribute 'ExifTags'

In [7]:

with open('./demo_material/ktvic.pickle', 'rb') as f:
        coco_tokens = pickle.load(f)
        sos_idx = coco_tokens['word2idx_dict'][coco_tokens['sos_str']]
        eos_idx = coco_tokens['word2idx_dict'][coco_tokens['eos_str']]


In [ ]:
class BaseConfig:
    """ Expansionnet_V2 model config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class ICConfig(BaseConfig):
  model_dim =  512
  N_enc = 3 
  N_dec = 3

  max_seq_len = 53



  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  load_path = 'checkpoint_2024-12-19-13_09_34_epoch0it440bs16_xe_.pth'

  beam_size = 5

  

In [2]:
from argparse import Namespace
drop_args = Namespace(enc=0.0,
                          dec=0.0,
                          enc_input=0.0,
                          dec_input=0.0,
                          other=0.0)

In [10]:
img_size = 384
model = End_ExpansionNet_v2(swin_img_size=img_size, swin_patch_size=4, swin_in_chans=3,
                                swin_embed_dim=192, swin_depths=[2, 2, 18, 2], swin_num_heads=[6, 12, 24, 48],
                                swin_window_size=12, swin_mlp_ratio=4., swin_qkv_bias=True, swin_qk_scale=None,
                                swin_drop_rate=0.0, swin_attn_drop_rate=0.0, swin_drop_path_rate=0.0,
                                swin_norm_layer=torch.nn.LayerNorm, swin_ape=False, swin_patch_norm=True,
                                swin_use_checkpoint=False,
                                final_swin_dim=1536,

                                d_model=model_dim, N_enc=N_enc,
                                N_dec=N_dec, num_heads=8, ff=2048,
                                num_exp_enc_list=[32, 64, 128, 256, 512],
                                num_exp_dec=16,
                                output_word2idx=coco_tokens['word2idx_dict'],
                                output_idx2word=coco_tokens['idx2word_list'],
                                max_seq_len=53, drop_args=drop_args,
                                rank=device)
checkpoint = torch.load(load_path, map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])
print("Model loaded ...")

Model loaded ...


In [ ]:
input_images = []